In [17]:
import polars as pl
import altair as alt

def parse_duration(col: str) -> pl.Expr:
    """
    Parse a duration string in the format 'XXd XXh XXmin XXs' into a Polars Duration type.
    
    Args:
        col: Name of the column containing duration strings.
    
    Returns:
        Polars expression that produces a Duration column.
    """
    return (
        pl.col(col)
                .str.extract_groups(r"(\d+)d\s*(\d+)h\s*(\d+)min\s*(\d+)s")
                .struct.rename_fields(["days", "hours", "minutes", "seconds"])
                .pipe(lambda x: pl.duration(
                    days=x.struct.field("days").cast(pl.Int64, strict=False),
                    hours=x.struct.field("hours").cast(pl.Int64, strict=False),
                    minutes=x.struct.field("minutes").cast(pl.Int64, strict=False),
                    seconds=x.struct.field("seconds").cast(pl.Int64, strict=False)
                ))
    )

In [21]:
pika_df = pl.read_csv("data/download.csv").with_columns(
    pl.col("Start").str.to_datetime(format="%d/%m/%Y %H:%M:%S"),
    pl.col("End").str.to_datetime(format="%d/%m/%Y %H:%M:%S"),
    parse_duration("Walltime").alias("Walltime"),
    parse_duration("Pending").alias("Pending"),
    parse_duration("Duration").alias("Duration"),
)

pika_df.sort("Pending")

shape: (189, 15)
┌──────────┬────────────┬────────────┬────────────┬───┬────────────┬───────┬───────────┬───────────┐
│ Job ID   ┆ Project    ┆ Start      ┆ End        ┆ … ┆ Duration   ┆ Core  ┆ Used      ┆ Partition │
│ ---      ┆ ---        ┆ ---        ┆ ---        ┆   ┆ ---        ┆ Hours ┆ Walltime  ┆ ---       │
│ i64      ┆ str        ┆ datetime[μ ┆ datetime[μ ┆   ┆ duration[μ ┆ ---   ┆ ---       ┆ str       │
│          ┆            ┆ s]         ┆ s]         ┆   ┆ s]         ┆ f64   ┆ f64       ┆           │
╞══════════╪════════════╪════════════╪════════════╪═══╪════════════╪═══════╪═══════════╪═══════════╡
│ 17341221 ┆ p_lv_inter ┆ 2025-06-02 ┆ 2025-06-02 ┆ … ┆ 30m 9s     ┆ 3.02  ┆ 100.5     ┆ barnard   │
│          ┆ net        ┆ 08:30:07   ┆ 09:00:16   ┆   ┆            ┆       ┆           ┆           │
│ 17336715 ┆ p_lv_inter ┆ 2025-05-30 ┆ 2025-05-30 ┆ … ┆ 30m 28s    ┆ 3.05  ┆ 101.56    ┆ barnard   │
│          ┆ net        ┆ 22:20:13   ┆ 22:50:41   ┆   ┆            ┆       ┆           ┆           │
│ 17324136 ┆ p_lv_inter ┆ 2025-05-29 ┆ 2025-05-29 ┆ … ┆ 30m 22s    ┆ 3.04  ┆ 101.22    ┆ barnard   │
│          ┆ net        ┆ 13:10:15   ┆ 13:40:37   ┆   ┆            ┆       ┆           ┆           │
│ 17323529 ┆ p_lv_inter ┆ 2025-05-28 ┆ 2025-05-28 ┆ … ┆ 19m 17s    ┆ 1.93  ┆ 64.28     ┆ barnard   │
│          ┆ net        ┆ 19:15:47   ┆ 19:35:04   ┆   ┆            ┆       ┆           ┆           │
│ 17299108 ┆ p_lv_inter ┆ 2025-05-26 ┆ 2025-05-26 ┆ … ┆ 24m 59s    ┆ 2.5   ┆ 83.28     ┆ barnard   │
│          ┆ net        ┆ 08:55:07   ┆ 09:20:06   ┆   ┆            ┆       ┆           ┆           │
│ …        ┆ …          ┆ …          ┆ …          ┆ … ┆ …          ┆ …     ┆ …         ┆ …         │
│ 17157301 ┆ p_lv_inter ┆ 2025-05-19 ┆ 2025-05-19 ┆ … ┆ 30m 8s     ┆ 3.01  ┆ 100.44    ┆ barnard   │
│          ┆ net        ┆ 16:20:00   ┆ 16:50:08   ┆   ┆            ┆       ┆           ┆           │
│ 17181314 ┆ p_lv_inter ┆ 2025-05-20 ┆ 2025-05-20 ┆ … ┆ 23m 16s    ┆ 2.33  ┆ 77.56     ┆ barnard   │
│          ┆ net        ┆ 11:33:33   ┆ 11:56:49   ┆   ┆            ┆       ┆           ┆           │
│ 17100910 ┆ p_lv_inter ┆ 2025-05-15 ┆ 2025-05-15 ┆ … ┆ 30m 5s     ┆ 3.01  ┆ 100.28    ┆ barnard   │
│          ┆ net        ┆ 16:43:08   ┆ 17:13:13   ┆   ┆            ┆       ┆           ┆           │
│ 17252348 ┆ p_lv_inter ┆ 2025-05-21 ┆ 2025-05-21 ┆ … ┆ 30m 15s    ┆ 3.02  ┆ 100.83    ┆ barnard   │
│          ┆ net        ┆ 17:19:46   ┆ 17:50:01   ┆   ┆            ┆       ┆           ┆           │
│ 17168791 ┆ p_lv_inter ┆ 2025-05-20 ┆ 2025-05-20 ┆ … ┆ 30m 14s    ┆ 3.02  ┆ 100.78    ┆ barnard   │
│          ┆ net        ┆ 02:49:12   ┆ 03:19:26   ┆   ┆            ┆       ┆           ┆           │
└──────────┴────────────┴────────────┴────────────┴───┴────────────┴───────┴───────────┴───────────┘

In [20]:
alt.Chart(pika_df.with_columns(pl.col("Pending").dt.total_seconds()).select(["Start", "Pending"])).mark_line().encode(
    x="Start",
    y=alt.Y("Pending", title="pending time in sec")
)


alt.Chart(...)